In [1]:
import os,sys, math, ROOT
from array import array
import numpy as np
# %jsroot on

#Importing a helper file where some functions are defined
from helper import *

#Input file
inputfile = ROOT.TFile("../Data/data.root","READ")

#Output file
outputfile = ROOT.TFile("output.root","RECREATE")

#Accessing the "TTree" from root that contains the data (one entry per event)
tree = inputfile.Get("ntuplizer/tree")

nentries = tree.GetEntries()
print("Number of entries: ", nentries)

Welcome to JupyROOT 6.26/06
Number of entries:  104317


In [2]:
hist_pT_2e = ROOT.TH1F("hist_pT_e",";Transverse momentum for the electron pair [GeV];Events", 500, 0, 340)
hist_pT_2m = ROOT.TH1F("hist_pT_m",";Transverse momentum for the muon pair [GeV];Events", 500, 0, 340)

In [3]:
#Loop over entries
#invariant_2e2u = np.array([])
for i in range(0,nentries):
    tree.GetEntry(i) # Load entry number i
    n_e,n_mu_minus,n_mu_p,n_p = 0,0,0,0   

    for lep in range(0,len(tree._lPt)): # Loop over leptons per event
        if tree._lpdgId[lep] == 11 and n_e != 1: # Detect if lepton == electron
            n_e += 1
            Index_e = lep
        elif tree._lpdgId[lep] == -11 and n_p != 1: # Detect if lepton == positron
            n_p += 1
            Index_p = lep
        elif tree._lpdgId[lep] == 13 and n_mu_minus != 1: # Detect if lepton == muon-
            n_mu_minus += 1
            Index_mu_minus = lep
        elif tree._lpdgId[lep] == -13 and n_mu_p != 1: # Detect if lepton == muon+
            n_mu_p +=1
            Index_mu_plus = lep

        if (n_e,n_p,n_mu_minus,n_mu_p) == (1,1,1,1): # If all sought-after leptons are detected, break out of the loop
            invariant_mass = invariantmass4l(tree,Index_e,Index_p,Index_mu_minus,Index_mu_plus)

            invariant_mass_e = invariantmass2l(tree,Index_e,Index_p)
            invariant_mass_mu = invariantmass2l(tree,Index_mu_minus,Index_mu_plus)

            if invariant_mass_e > 4 and invariant_mass_mu > 4:
                pT_e = tree._lPt[Index_e]
                pT_p = tree._lPt[Index_p]
                pT_mu_p = tree._lPt[Index_mu_plus]
                pT_mu_m = tree._lPt[Index_mu_minus]

                hist_pT_2e.Fill(pT_e)
                hist_pT_2e.Fill(pT_p)
                hist_pT_2m.Fill(pT_mu_m)
                hist_pT_2m.Fill(pT_mu_p)

In [4]:
savehisto(outputfile,hist_pT_2e,"hist_pT_2e",logy=True)
savehisto(outputfile,hist_pT_2m,"hist_pT_2m",logy=True)

Info in <TCanvas::Print>: pdf file pdfs/hist_pT_2e.pdf has been created
Info in <TCanvas::Print>: pdf file pdfs/hist_pT_2m.pdf has been created
